run block with #execute to execute certain function
run block with #check to check the certain property 

In [1]:
#execute
# add the path and load the module
#(I haven't write the setup.py)
import os 
import sys
import DPMacro.AFill as af


In [10]:
#split frames
from DPMacro.util import read_in,write_out
# from itertools import enumerate
def split_frame(file:str)->None:
    s=read_in(file)
    if len(s)<=1:
        raise ValueError
    outdir=file.replace('.pdb','')
    if not os.path.isdir(outdir):
        os.mkdir(outdir)
    for i,frame in enumerate(s):
        write_out(frame,os.path.join(outdir,f'{i}.pdb'))
split_frame('/root/DPMacro/tutorial_AFill/1ei2.pdb')




In [22]:
#execute
#load the model(object) structure and the rev structure 
aligner=af.structure_aligner(rna=True)
# aligner.load_structure('/root/DPMacro/tutorial_AFill/1ei2/0.pdb')
aligner.load_reference('/root/DPMacro/tutorial_AFill/1ei2/1.pdb')

In [24]:
#check
#check the fasta information
# aligner.sequences
aligner.reference_sequences

{'A': 'GGCAGUGUGAGUACCUUCACACGUC'}

In [7]:
#check
#chain distance may give you some insight on chain's homology
# aligner.chain_distance


In [25]:
#check the ligand list
aligner.ligands

[<Residue NMY het=H_NMY resseq=26 icode= >]

In [26]:
#check
#residue in aligner.ligands is the same residue in aligner.reference
aligner.reference[0]['A'][('H_NMY', 26, ' ')] is aligner.ligands[0]

True

In [27]:
#execute
#set the ligand of concern fro down stream work
aligner.set_ligand(aligner.ligands[0])

In [30]:
#check
aligner.neighbor

[<Residue A het=  resseq=21 icode= >,
 <Residue G het=  resseq=5 icode= >,
 <Residue C het=  resseq=18 icode= >,
 <Residue U het=  resseq=8 icode= >,
 <Residue G het=  resseq=11 icode= >,
 <Residue C het=  resseq=14 icode= >,
 <Residue U het=  resseq=17 icode= >,
 <Residue G het=  resseq=7 icode= >,
 <Residue C het=  resseq=20 icode= >,
 <Residue A het=  resseq=4 icode= >,
 <Residue A het=  resseq=10 icode= >,
 <Residue A het=  resseq=13 icode= >,
 <Residue A het=  resseq=19 icode= >,
 <Residue C het=  resseq=3 icode= >,
 <Residue U het=  resseq=16 icode= >,
 <Residue C het=  resseq=22 icode= >,
 <Residue U het=  resseq=6 icode= >,
 <Residue G het=  resseq=9 icode= >,
 <Residue C het=  resseq=15 icode= >]

In [42]:
from DPMacro.util import integrated_residue_iterator,allowed_residue_source,CHAIN_HOLDER
from Bio.PDB.Chain import Chain
from Bio.PDB.Model import Model
def write_out_segments(residues:allowed_residue_source):
    residue_list=list(integrated_residue_iterator(residues))
    residue_list.sort(key=lambda x :-ord(x.get_parent().id)*10000+x.id[1])
    model=Model('0')
    for residue in residue_list:
        chainid=residue.get_parent().id
        if chainid not in model:
            new_chain=Chain(chainid)
            model.add(new_chain)
        model[chainid].add(residue.copy())
    return model



In [44]:
write_out(write_out_segments(aligner.neighbor),'pocket.pdb')
write_out(write_out_segments(aligner.selected_ligand),'ligand.pdb')

In [39]:
ord('A')

65

In [32]:
'A' in aligner.reference[0]

True

function 1: do not know the homology chain ?
run  aligner.transform() directly,
it will try all chain combination and return the combination with lowest pocket rmsd.
it will print: combination, number of hit residue,rmsd (full &pocket) 
and return: best chain combine; rmsd (full &pocket) 

then run aligner.save_align() to save the pdbs.

Warning!
you can see that since cmd.align will drop a few atoms in each alignment cycle,
wrong chain hit can have as nice behavior as the 'right one'
going to be debugged. 

In [19]:
aligner.transform()

('A', 'B')	12	(0.9076577425003052, 2.1527609825134277)	('A', 'C')	12	(0.9076577425003052, 2.1527609825134277)	('A', 'D')	12	(0.9076577425003052, 2.1527609825134277)	('A', 'E')	12	(0.9076577425003052, 2.1527609825134277)	('B', 'A')	20	(0.9076577425003052, 0.4204222559928894)	('B', 'C')	20	(0.9076577425003052, 0.4204222559928894)	('B', 'D')	20	(0.9076577425003052, 0.4204222559928894)	('B', 'E')	20	(0.9076577425003052, 0.4204222559928894)	('C', 'A')	20	(0.9076577425003052, 0.422482430934906)	('C', 'B')	20	(0.9076577425003052, 0.422482430934906)	('C', 'D')	20	(0.9076577425003052, 0.422482430934906)	('C', 'E')	20	(0.9076577425003052, 0.422482430934906)	('D', 'A')	20	(0.9076577425003052, 6.543617248535156)	('D', 'B')	20	(0.9076577425003052, 6.543617248535156)	('D', 'C')	20	(0.9076577425003052, 6.543617248535156)	('D', 'E')	20	(0.9076577425003052, 6.543617248535156)	('E', 'A')	20	(0.9076577425003052, 6.539333820343018)	('E', 'B')	20	(0.9076577425003052, 6.539333820343018)	('E', 'C')	20	(0.907

(('B', 'A'), (0.9076577425003052, 0.4204222559928894))

In [24]:
aligner.save_align()

(0.9076577425003052, 0.4204222559928894)

function 2: sure about the chain map?
set aligner.mapped_chain,and run aligner.save_align directly

In [28]:
aligner.mapped_chain=('B','D')
aligner.save_align(ligandfile='bdligand.pdb',
                    ref_pocketfile='bdaligned_pocket.pdb',
                    obj_pocketfile='bdmodel_pocket.pdb')

(0.9076577425003052, 0.4204222559928894)